In [ ]:
# Cell 1: Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
import time 
from tqdm import tqdm # For a nice progress bar

# Add the 'src' folder to the path
sys.path.insert(0, os.path.abspath('../src'))

# --- Import our two classes ---
from models.gbm import GeometricBrownianMotion
from calibration.gbm_calibrator import GbmCalibrator

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
print("All modules imported successfully.")

All modules imported successfully.


In [ ]:
# Cell 2: Setup the Monte Carlo Test
# These are the "true" parameters we want to find
true_gbm_params = {
    'S0': 100,
    'mu': 0.05,       # 5% total drift
    'sigma': 0.1,     # 20% diffusion vol
}

# --- Simulation setup ---
T = 10       # 1 year of data for each test
dt = 1/252    # Daily steps
n_test_paths = 500 # Run 500 tests to get a good average

# --- Result storage ---
calibrated_mus = []
calibrated_sigmas = []

# --- Instantiate our models once ---
gbm_model = GeometricBrownianMotion(**true_gbm_params)
gbm_cal = GbmCalibrator(dt=dt)

print(f"Running {n_test_paths} GBM calibration tests...")
print(f"True Params: {true_gbm_params}")

# Cell 3: Run the Test Loop
start_time = time.time()

# tqdm adds a progress bar
for _ in tqdm(range(n_test_paths)):
    # 1. Simulate one new, random path
    sim_path_array = gbm_model.simulate(T, dt, n_paths=1).flatten()
    sim_series = pd.Series(sim_path_array)

    # 2. Calibrate this one path
    # (GBM calibration is stable, so no need for try/except)
    params = gbm_cal.fit(sim_series)
    
    # 3. Store the results
    calibrated_mus.append(params['mu'])
    calibrated_sigmas.append(params['sigma'])

end_time = time.time()
print(f"--- Test Complete in {end_time - start_time:.2f} seconds ---")

# Cell 4: Analyze and Display Results
# 1. Get the average of each parameter column
mean_mu = np.mean(calibrated_mus)
mean_sigma = np.mean(calibrated_sigmas)

# 2. Get the standard deviation of our estimates
std_mu = np.std(calibrated_mus)
std_sigma = np.std(calibrated_sigmas)

# 3. Print a clean comparison table
print("\n=== True vs Average Calibrated Parameters ===")
print(f"{'Parameter':<16}{'True Value':>15}{'Average Calibrated':>25}")
print("-" * 56)
print(f"{'μ (drift)':<16}{true_gbm_params['mu']:>15.4f}{mean_mu:>25.4f}")
print(f"{'σ (diff vol)':<16}{true_gbm_params['sigma']:>15.4f}{mean_sigma:>25.4f}")

# 4. Print the standard deviation of the estimates
print("\n=== Estimation Standard Deviations ===")
print(f"{'Parameter':<16}{'Std Dev (across runs)':>30}")
print("-" * 46)
print(f"{'μ (drift)':<16}{std_mu:>30.4f}")
print(f"{'σ (diff vol)':<16}{std_sigma:>30.4f}")

Running 500 GBM calibration tests...
True Params: {'S0': 100, 'mu': 0.05, 'sigma': 0.1}


100%|████████████████████████████████████████| 500/500 [00:03<00:00, 148.97it/s]

--- Test Complete in 3.36 seconds ---

=== True vs Average Calibrated Parameters ===
Parameter            True Value       Average Calibrated
--------------------------------------------------------
μ (drift)                0.0500                   0.0534
σ (diff vol)             0.1000                   0.1000

=== Estimation Standard Deviations ===
Parameter                Std Dev (across runs)
----------------------------------------------
μ (drift)                               0.0305
σ (diff vol)                            0.0014
